<a href="https://colab.research.google.com/github/carloseducorinto/Agents/blob/main/L7_job_application_crew_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [30]:
#!pip install -q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
#!pip install langchain_groq


In [31]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [32]:
from crewai import Agent, Crew, Task
import os
from google.colab import userdata
from langchain_groq import ChatGroq
from crewai import Agent, Task, Crew

In [33]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
#os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
llm=ChatGroq(temperature=0, model_name="llama3-70b-8192", api_key=userdata.get("GROQ_API_KEY"))

#llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768",api_key=userdata.get("GROQ_API_KEY"))

## crewAI Tools

In [34]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')



In [35]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

John A. Smith

Contact Information:

Email: john.smith@example.com
Phone: (123) 456-7890
LinkedIn: linkedin.com/in/johnasmith
Address: 123 Tech Street, Innovation City, USA
Professional Summary:

Dynamic and results-oriented IT professional with over 10 years of experience specializing in Artificial Intelligence and Machine Learning. Proven track record in developing innovative AI solutions, enhancing business processes, and driving strategic technological initiatives. Adept at leveraging data science methodologies to solve complex problems and improve decision-making.

Technical Skills:

Programming Languages: Python, Java, R, SQL, TensorFlow, PyTorch
AI/ML Tools: Keras, Scikit-learn, NLP, OpenCV, Hadoop, Spark
Data Visualization: Tableau, Power BI, Matplotlib, Seaborn
Cloud Platforms: AWS, Azure, Google Cloud
Databases: MySQL, PostgreSQL, MongoDB
Other: Git, Docker, Kubernetes, Agile/Scrum
Professional Experience:

Senior AI Engineer
Tech Innovators Inc., Innovation City, USA
January 2020 – Present

Led a team of AI engineers in the development of predictive models and algorithms to optimize supply chain management, resulting in a 20% reduction in operational costs.
Developed and deployed machine learning models for customer behavior analysis, increasing customer retention by 15%.
Implemented natural language processing (NLP) techniques to enhance chatbots and virtual assistants, improving customer service response times by 30%.
Collaborated with cross-functional teams to integrate AI solutions into existing systems, ensuring seamless implementation and scalability.
AI/ML Specialist
Data Solutions Corp., Data City, USA
June 2015 – December 2019

Designed and implemented machine learning models for fraud detection, reducing fraudulent activities by 25%.
Conducted extensive data analysis and feature engineering to improve model accuracy and performance.
Developed automated data pipelines and workflows to streamline the data processing and model training processes.
Provided technical guidance and mentorship to junior data scientists and engineers.
Machine Learning Engineer
InnovateTech, Tech Town, USA
August 2012 – May 2015

Built and deployed machine learning algorithms for predictive maintenance, decreasing equipment downtime by 18%.
Collaborated with data engineers to ensure the quality and integrity of data used for model training and evaluation.
Conducted A/B testing and model validation to assess the effectiveness of AI solutions.
Assisted in the development of company-wide AI strategy and roadmap.
Education:

Master of Science in Computer Science
University of Technology, Tech Town, USA
Graduated: May 2012

Bachelor of Science in Information Technology
Innovation University, Innovation City, USA
Graduated: May 2010

Certifications:

Certified Artificial Intelligence Practitioner (CAIP)
AWS Certified Machine Learning – Specialty
Microsoft Certified: Azure AI Engineer Associate
Professional Affiliations:

Member, Association for the Advancement of Artificial Intelligence (AAAI)
Member, International Association for Artificial Intelligence (IAAI)
Projects:

AI-Powered Recommendation System: Developed a recommendation engine for an e-commerce platform, increasing sales by 22%.
Automated Image Recognition System: Created an image classification system using deep learning, achieving 95% accuracy in identifying defects in manufacturing processes.
Predictive Analytics Dashboard: Designed a real-time analytics dashboard for monitoring and predicting sales trends, providing actionable insights for the sales team.
Soft Skills:

Strong analytical and problem-solving abilities
Excellent communication and teamwork skills
Ability to manage multiple projects and meet deadlines
Creative and innovative thinking
Detail-oriented and highly organized

## Creating Agents

In [36]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm = llm,
)

In [37]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm = llm,
)

In [38]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm = llm,
)

In [39]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm = llm,
)

## Creating Tasks

In [40]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True,
    llm = llm,
)

In [41]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True,
    llm = llm,
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [42]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist,
    llm = llm,
)

In [43]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer,
    llm = llm
)


## Creating the Crew

In [44]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [45]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/carloseducorinto',
    'personal_writeup': """Carlos Eduardo is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Carlos Eduardo has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [46]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 



> Entering new CrewAgentExecutor chain...

 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/carloseducorinto) URLs, and personal write-up (Carlos Eduardo is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Carlos Eduardo has succes

Exception in thread Thread-19 (_execute):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
  File "/usr/local/lib/python3.10/dist-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
  File "/usr/local/lib/python3.10/dist-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.10/dist-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/usr/local/lib/python3.10/dist-packages/crewai/agents/executor.py", line 124, in _call
    next_step_output = self._take_next_step(
  File "/usr/local/lib/python3.10/dist-packages/langchain/agents/agent.py", line 1138

The URL "https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund" is a valid URL.

Thought:
Now that I have confirmed the URL is valid, I can use the "Read website content" tool to extract job requirements from the job posting.

Action: Read website content
Action Input: {'website\_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund'}
 

I encountered an error while trying to use the tool. This was the error: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?.
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: I need to double-check the website URL for any potential issues.

Action: Search the internet
Action Input: {'search\_query': 'validate URL "https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lev

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hyrtxkjwfs18em25ps2nfqe8` on tokens per minute (TPM): Limit 6000, Used 0, Requested ~6085. Please try again in 850ms. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [47]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

Here is the updated resume to highlight the candidate's transferable skills and experiences that align with the job requirements:

John A. Smith

Contact Information:

Email: john.smith@example.com
Phone: (123) 456-7890
LinkedIn: linkedin.com/in/johnasmith
Address: 123 Tech Street, Innovation City, USA

Professional Summary:

Dynamic and results-oriented IT professional with over 10 years of experience specializing in Artificial Intelligence and Machine Learning. Proven track record in developing innovative AI solutions, enhancing business processes, and driving strategic technological initiatives. Adept at leveraging data science methodologies to solve complex problems and improve decision-making.

Technical Skills:

Programming Languages: Python, Java, R, SQL, TensorFlow, PyTorch
AI/ML Tools: Keras, Scikit-learn, NLP, OpenCV, Hadoop, Spark
Data Visualization: Tableau, Power BI, Matplotlib, Seaborn
Cloud Platforms: AWS, Azure, Google Cloud
Databases: MySQL, PostgreSQL, MongoDB
Other: Git, Docker, Kubernetes, Agile/Scrum

Professional Experience:

Senior AI Engineer
Tech Innovators Inc., Innovation City, USA
January 2020 – Present

Led a team of AI engineers in the development of predictive models and algorithms to optimize supply chain management, resulting in a 20% reduction in operational costs.
Developed and deployed machine learning models for customer behavior analysis, increasing customer retention by 15%.
Implemented natural language processing (NLP) techniques to enhance chatbots and virtual assistants, improving customer service response times by 30%.
Collaborated with cross-functional teams to integrate AI solutions into existing systems, ensuring seamless implementation and scalability.

Transferable Skills and Experiences:

* Strong analytical and problem-solving abilities
* Excellent communication and teamwork skills
* Ability to manage multiple projects and meet deadlines
* Creative and innovative thinking
* Detail-oriented and highly organized
* Experience in developing and deploying machine learning models
* Strong data analysis and feature engineering skills
* Ability to work with cross-functional teams and integrate AI solutions into existing systems

Education:

Master of Science in Computer Science
University of Technology, Tech Town, USA
Graduated: May 2012

Bachelor of Science in Information Technology
Innovation University, Innovation City, USA
Graduated: May 2010

Certifications:

Certified Artificial Intelligence Practitioner (CAIP)
AWS Certified Machine Learning – Specialty
Microsoft Certified: Azure AI Engineer Associate

Professional Affiliations:

Member, Association for the Advancement of Artificial Intelligence (AAAI)
Member, International Association for Artificial Intelligence (IAAI)

Projects:

AI-Powered Recommendation System: Developed a recommendation engine for an e-commerce platform, increasing sales by 22%.
Automated Image Recognition System: Created an image classification system using deep learning, achieving 95% accuracy in identifying defects in manufacturing processes.
Predictive Analytics Dashboard: Designed a real-time analytics dashboard for monitoring and predicting sales trends, providing actionable insights for the sales team.

In [48]:
display(Markdown("./interview_materials.md"))

./interview_materials.md